In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import mmap
import random
import pickle
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# Hyperparameters
batch_size = 64
block_size = 128
max_iters = 200
lr = 3e-4
eval_iters = 100
num_embeddings = 384
num_heads = 4
num_layers = 4
dropout = 0.2

In [3]:
chars = ""
with open('E:/Saathvik/openwebtext/vocab.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    chars = sorted(list(set(text)))
vocab_size = len(chars)

In [4]:
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [5]:
def get_random_chunk(split):
    filename = "E:/Saathvik/openwebtext/train_split.txt" if split == 'train' else "E:/Saathvik/openwebtext/test_split.txt"
    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            file_size = len(mm)
            start = random.randint(0, file_size - (block_size * batch_size)) # random starting position
            mm.seek(start)
            block = mm.read(block_size * batch_size - 1) # read a block of data
            decoded = block.decode('utf-8', errors='ignore').replace('\r', '') # decode and clean the data
            data = torch.tensor(encode(decoded), dtype=torch.long, device=device) # encode and convert to tensor
    return data         

In [6]:
def get_batch(split):
    data = get_random_chunk(split)
    xi = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i + block_size] for i in xi])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in xi])
    x, y = x.to(device), y.to(device)
    return x, y

In [7]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval() # type: ignore
    for split in ['train', 'test']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y) # type: ignore
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train() # type: ignore
    return out

In [8]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(num_embeddings, head_size, bias=False) # linear transformation for keys
        self.query = nn.Linear(num_embeddings, head_size, bias=False) # linear transformation for queries
        self.value = nn.Linear(num_embeddings, head_size, bias=False) # linear transformation for values
        # lower triangular matrix for masking future tokens
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size))) 
        self.dropout = nn.Dropout(dropout) 
    
    def forward(self, x):
        # input x: (B, T, C) where B=batch size, T=block size, C=channels
        # output: (B, T, C) where C=head_size
        B, T, C = x.shape 
        k = self.key(x) # (B, T, head_size)
        q = self.query(x) #@ (B, T, head_size)
        
        # compute weighted attention scores
        weights = q @ k.transpose(-2, -1) * k.shape[-1] ** -0.5 # scaled dot-product attention
        weights = weights.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # mask future tokens
        weights = F.softmax(weights, dim=-1) # softmax to get attention weights
        weights = self.dropout(weights)
        v = self.value(x) # (B, T, head_size)
        output = weights @ v # matrix multiplication
        return output

In [9]:
# MultiHead Attention Module 
# (scaled dot-product attention (k, q, v) -> concatenate results -> linear projection)
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.projection = nn.Linear(head_size * num_heads, num_embeddings)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        output = torch.cat([h(x) for h in self.heads], dim=-1)  # concatenate results from all heads
        output = self.dropout(self.projection(output))  # linear projection and dropout
        return output

In [10]:
# transformer block feed forward class (linear -> ReLU -> linear)
class FeedForward(nn.Module):
    def __init__(self, num_embeddings):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(num_embeddings, 4 * num_embeddings),
            nn.ReLU(),
            nn.Linear(4 * num_embeddings, num_embeddings),
            nn.Dropout(dropout), # dropout for regularization (prevent overfitting)
        )

    def forward(self, x):
        return self.net(x)

In [11]:
# transformer block class (MultiHeadAttention -> FeedForward -> LayerNorm -> LayerNorm)
class Block(nn.Module):
    def __init__(self, num_embeddings, num_heads):
        super().__init__()
        head_size = num_embeddings // num_heads
        self.sa = MultiHeadAttention(num_heads, head_size)
        self.ffwd = FeedForward(num_embeddings)
        self.ln1 = nn.LayerNorm(num_embeddings)
        self.ln2 = nn.LayerNorm(num_embeddings)
    
    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)  # residual connection
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x      

In [12]:
# basic GPT language model
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, num_embeddings)
        self.position_embedding_table = nn.Embedding(block_size, num_embeddings)
        self.blocks = nn.Sequential(*[Block(num_embeddings, num_heads) for _ in range(num_layers)])
        self.ln_f = nn.LayerNorm(num_embeddings) # final layer normalization
        self.lm_head = nn.Linear(num_embeddings, vocab_size) # final linear layer to project to vocab size
        self.apply(self.__init__weights) # initialize weights
    
    def __init__weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    
    def forward(self, idx, targets=None):
        # B=batch size, T=time, C=channels (can be vocab size)
        B, T = idx.shape
        token_embeddings = self.token_embedding_table(idx)  # (B, T, C)
        position_embeddings = self.position_embedding_table(torch.arange(T, device=device))  # (T, C)
        x = token_embeddings + position_embeddings  # (B, T, C)
        x = self.blocks(x)  # feed through transformer blocks
        x = self.ln_f(x) # final layer normalization
        logits = self.lm_head(x)  # project to vocab size (B, T, vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape 
            
            # reshape logits and targets for cross-entropy loss
            logits = logits.view(B*T, C) 
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets) 
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(idx[:, -block_size:])  # get predictions
            logits = logits[:, -1, :] # (B, C) last time step
            probs = F.softmax(logits, dim=-1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [18]:
model = GPTLanguageModel(vocab_size).to(device)

with open('gpt_v1.pkl', 'rb') as f:
    model= pickle.load(f)
print("Model loaded from gpt-v1.pkl")

Model loaded from gpt-v1.pkl


In [19]:
#PyTorch Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f'step: {iter}, train loss: {losses["train"]:.4f}, test loss: {losses["test"]:.4f}')
    xb, yb = get_batch('train')
    
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward() # type: ignore
    optimizer.step()
print(loss.item()) # type: ignore

# save the model
with open('gpt_v1.pkl', 'wb') as f:
    pickle.dump(model, f)
print("Model saved to gpt_v1.pkl")

step: 0, train loss: 2.3077, test loss: 2.2611
step: 100, train loss: 2.2828, test loss: 2.2806
2.1401333808898926
Model saved to gpt_v1.pkl


In [26]:
model = GPTLanguageModel(vocab_size).to(device)

for i in range(3):
    print(f"Iteration {i+1}")
    with open('gpt_v1.pkl', 'rb') as f:
        model = pickle.load(f)
    print("Model loaded from gpt-v1.pkl")
    
    #PyTorch Optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr)

    for iter in range(max_iters):
        if iter % eval_iters == 0:
            losses = estimate_loss()
            print(f'step: {iter}, train loss: {losses["train"]:.4f}, test loss: {losses["test"]:.4f}')
        xb, yb = get_batch('train')
        
        logits, loss = model.forward(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward() # type: ignore
        optimizer.step()
    print(loss.item()) # type: ignore
    
        # save the model
    with open('gpt_v1.pkl', 'wb') as f:
        pickle.dump(model, f)
    print("Model saved to gpt_v1.pkl")

Iteration 1
Model loaded from gpt-v1.pkl
step: 0, train loss: 1.6278, test loss: 1.7506


KeyboardInterrupt: 

In [25]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)  # initial context
generated_chars = decode(model.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)   

                                                                                                                                                                  Hou repen que lon Belt on Br Hon 25 vor-Fientwore Clays ris hem placcesion.


5 conding only on were bat a simplease, prewecin servscoused to battimes were said, the rightifieght. Commocnity ito anime a wild hast bey they are highanda Few 2001 head is astincersumgent Commediont and womey erthouthd compioling socumpon for a pronerientroce
